# Workshop 1: excersices for practice

This workshops 1 problems are not assessed and are intended for practicing the the concepts introduced in the 1st ML lecture, the and the notebooks `data-science-tools.ipynb` and `lecture2.ipynb`. 

It is recommended that you go thorugh those notebooks 1st and try to understand each line before jumping into the excersises.

*Responsible:* Robert Currie (<rob.currie@ed.ac.uk>) borrowed from Guillermo Hamity

## Wine quality dataset

The [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine%20quality) will be analysed

***Before*** we begin you will need to copy the dataset `wine_quality.csv` into the same directory as you're running this notebook!

### Dataset information
Two datasets were created, using red and white wine samples.
The inputs include objective tests (e.g. PH values) and the output is based on sensory data
(median of at least 3 evaluations made by wine experts). Each expert graded the wine quality 
between 0 (very bad) and 10 (very excellent). 
  
1. Number of Instances: red wine - 1599; white wine - 4898. 

2. Number of Attributes: 11 + output attribute
  
   Note: several of the attributes may be correlated, thus it makes sense to apply some sort of
   feature selection.

3. Attribute information:

   For more information, read [Cortez et al., 2009].

   Input variables (based on physicochemical tests):
   1. fixed acidity
   
   2. volatile acidity
   
   3. citric acid
   
   4. residual sugar
   
   5. chlorides
   
   6. free sulfur dioxide
   
   7. total sulfur dioxide
   
   8. density
   
   9. pH
   
   10. sulphates
   
   11. alcohol
   
   12. type (R=red, W=white)
   
   Output variable (based on sensory data): 
   12 - quality (score between 0 and 10)

### Purpose:

We will design a model which predicts the quality of **RED** wine based on the input features. In order to achive this we will perform some initial investigations, preperation, and visiualisation of the data. We will then train decition trees to distinguish good vs bad  **red** wines, and check the validity of our model on a testing dataset.

### Working through the problems
* Try and calculate the answers provided. 
* Keep code understandable and reproducible, i.e. the notebook can be **restarted** and **re-run**. You will need to do this when work is marked, so better to start practicing.
* If you are unsure on how to proceed please reference the two companion notebooks for relevant examples, or **ask one of the TAs**

### Notes on solution
* This CP exercise sheet is divided into three sections, corresponding to parts of the lecture. The problems will **not** be marked.
  - Data science tools 
  - Visualisation 
  - Decision tree classification + ensemble methods
* Exercises 1-5 can be solved with one line of code while still being legible. _(That's not a weird flex, nor required, just letting you know that if you find yourself with a 20-line long solution, it can probably be done in a simpler way.)_
* The following `pandas.DataFrame` functions may be useful: `head`, `describe`, `loc`, `query`, `unique`, `value_counts`, `sort_values`, `group_by`, `apply`, `transform`.

In [ ]:
#Turn off some warnings which we can ignore for this example
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Standard import(s)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn import datasets

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Load wine dataset
obs = pd.read_csv('wine_quality.csv')

In [ ]:
# Show available properties of the dataset
obs.keys()

In [ ]:
# Show dataset description
obs.head()

In [ ]:
# Drop duplicate measurements -- makes sense in this case, since measurements 
# with identical rows would imply identical wines (at least in composition),
# which would be double counted in the dataset
obs = obs.drop_duplicates()

---
## Data science tools 

This section covers exercises on data handling using pandas.DataFrames.
###### 1. How many observations have been collected and how many features are listed per observation? 

In [ ]:
obs.count(numeric_only=True)

###### 2. How may wines are of each type, Red  and White?

In [ ]:
obs["type"].value_counts()

###### 3. What are the minimum and maximum wine qualities in the dataset ? 


In [ ]:
obs.groupby("type")["quality"].describe()

###### 4. Which type of wine (red or white) has the higher score on average ? 
_Hints:_
1. Group the data by type, ie, white and red wine
2. Get the mean of the quality for each type


In [ ]:
obs.groupby("type")["quality"].describe()

#### 5. How many measurements are there for each quality value per wine type?
Hits:
- group the data by type and quality
- count the values in each group



In [ ]:
obs.groupby("type")["quality"].count()

#### 6. Data preperation

From the investigation above we can see varying statistics between the red and white wines. Lets prepare the data in such a way that we define good and bad quality wines based on the median value of each wine type.

Make two datasets named `white` and `red` by doing the following

1. goup the data into Red and White wine
2. find the median values of quality for red and white wines seperately
3. create a new quality variable `good_qual` which is 0 for bad (less than the median) and 1 for good (more than or equal to the median)
4. drop columns `quality` and `type` from the datasets as they are no longer needed. Use the `drop` method


In [ ]:
# Find median value
red_median = obs.groupby("type").median()["quality"]['R']
white_median = obs.groupby("type").median()["quality"]['W']

# make two seperate dataframes
red = obs[ obs["type"] == 'R' ]
white = obs[ obs["type"] == 'W' ]

# Create Series constaining the good_qual feature for each sample
good_qual_red = red["quality"] >= red_median
good_qual_white = white["quality"] >= white_median


# Include good_qual featuire in the corresponding dataframe
red["good_qual"] = good_qual_red.astype(int)
white["good_qual"] = good_qual_white.astype(int)

# Drop the quality collums
red = red.drop(columns=["quality", "type"])
white = white.drop(columns=["quality", "type"])


---
## From here on we will be focusing on the **red** wine dataset

--------------------------------------------------------
###### 7.  Use the describe method to show statistics per good/bad wine in the red wine dataset for alcohol and sulphates

In [ ]:
features = ["alcohol", "sulphates"]
red.groupby("good_qual")[features].describe()

---
## Visualisation 
---
This section covers 2 exercises on visualisation using `matplotlib.pyplot`. 

###### 8 Make a correlation plot of all the features in the red wine dataset
- Use the porvided colormap
- Use the sns.heatmap function as shown in the lecture
- set the tick labels to represent the feature names


In [ ]:
# Use the following courmap in as the cmap parameter of sns.heatmap:
colormap = sns.diverging_palette(220, 10, as_cmap=True)

In [ ]:
red_corrolation = red.corr()
sns.heatmap(red_corrolation, annot=False, cmap=colormap)

##### 9 Pick the top five most correlated features to chlorides and make a pairplot split by good_qual

- Do not include "good_qual" in the calculation of most correlated
- Negative and positive correlations both show strong correlation (or anti correlation). Use the ablsolute value to pick the most correlated features

In [ ]:
red_corrolation[red_corrolation["chlorides"].abs() < 1.0]["chlorides"].abs().sort_values(ascending=False)

In [ ]:
top_correlations = ["alcohol", "density", "total_sulfur_dioxide", "citric_acid", "free_sulfur_dioxide"]

top_correlations_dataframe = red[top_correlations + ["good_qual"]]
sns.pairplot(top_correlations_dataframe, hue="good_qual")

#### 8. Make a scatter plot of the two most correlated features.
Example solution:

- Find the two most correlated features (well you can read them from the plot above, but try find them yourself by)
  - Mask diagonal points in the |correlation| matrix
  - Unstack the dataframe into a pandas Series
  - sort the values in decending order, then pick the top value (or use idxmax)
- Make the scatter plot using the ax.scatter method.
  - Make the distributions legible, either by setting the opacity (alpha=...).
  - Label X axis, and Y axis including units.
  - Include a legend two wine types.
- Discuss the figure. Do you think using the both variables to distinguish wine quality is usefull?  



In [ ]:
mask = red_corrolation.abs().unstack() < 1.0
sorted_correlations = red_corrolation.unstack()[mask].abs().sort_values(ascending=False).drop_duplicates()

fig, (ax1, ax2) = plt.subplots(2, 1)
features = sorted_correlations.index[:2]
print(features)
axes = [ax1, ax2]
markers = ["x", "x"]

for i, axis in enumerate(axes):
    for j, t in enumerate([0, 1]):
        mask = red["good_qual"] == t
        axis.scatter(red[features[i][0]], red[features[i][1]], label=t, marker=markers[j], s=0.5, alpha=0.5)
        pass
    # Draw legend
    axis.legend()
    axis.set_xlabel(features[i][0])
    axis.set_ylabel(features[i][1])

fig.suptitle("Highly correlated examples")
fig.tight_layout()
plt.show()



#### 9. Prepare a smaller pandas.DataFrame for visualisation and classification 

In the next sections, on decision trees and ensemble methods, we will try to classify the wine quality --- based on a subset of the available features. We also introduce labels for Poor and Good wines for plotting purposes:


In [ ]:
# define data and features
features = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
target   = ['good_qual']
wtype = ["Poor","Good"]


#### Therefore:

* Create a pandas.DataFrame, called dataset, which contains only the desired features and the target variable;
* Drop duplicates and NaN-values;
* Shuffle the dataframe;
* Create a pandas.DataFrame, called frames, containing the first 1000 observations of the dataset for each class
* Create a sample dataset by doing the concat of frames
* From sample panda dataframe convert into a numpy array for the inputs (called X) and target (called y);
* Flatten the converted y array; and
* Check their shapes to make sure they look alright.



In [ ]:
# Define dataset + remove duplicates and NAN
dataset = red[features + target].drop_duplicates().dropna()

# Define framge with shuffled 1000 samples or each type of quality
frames_good_quality = dataset[ dataset["good_qual"] == 1 ].sample(n=1000)
frames_bad_quality = dataset[ dataset["good_qual"] == 0 ].sample(n=1000)

# Create sample dataset with shuffled data
sample_dataset = pd.concat([frames_bad_quality, frames_good_quality])

# From frames create sample inputs (x) and targets (y)
x = np.asarray(sample_dataset[features])
y = np.asarray(sample_dataset[target]).flatten()

print(f"Shape of x is {x.shape} \nShape of y is {y.shape}")

## Decision tree classification 
---
This section covers 3 exercises on the use and understanding of decision trees and ensemble methods.

#### 10. Fit a decision tree classifier and plot the decision

* Split dataset into training set and test set (x_train, x_test, y_train, y_test)
* Create a `sklearn.tree.DecisionTreeClassifier` and set the maximum tree depth to 4, use `entropy` as the quality criterion.
* Fit the decision tree classifier
* Predict the response for the test and training datasets
* Print the accuracy of test and training datasets
* Make the decision tree graph and save it in png
* Find the important feature in the decision tree (use feature_importances method in the decision tree). Why do you think this feature is of particular importance in distinguishing good and poor wines?

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import export_graphviz
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics # Import scikit-learn metrics module for accuracy calculation
from sklearn import preprocessing # Import preprocessing for String-Int conversion

In [ ]:
# HYPER PARAMETERS
MAX_DEPTH = 4

# Split data int training and test dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

# Define basic decision tree object
# ASK WHAT TO SET AS MIN_SAMPLE AND MIN_SAMPLE_LEAF
decision_tree = DecisionTreeClassifier(criterion="entropy", splitter="best", max_depth=MAX_DEPTH)

# Train the decition tree
decision_tree.fit(x_train, y_train)

# Create predicted data using the test dataset
y_predicted_test = decision_tree.predict(x_test)
y_predicted_train = decision_tree.predict(x_train)

# Print accuracy of the model
print(f"Accuracy of basic decision tree w/ test dataset:\t {metrics.accuracy_score(y_true=y_test, y_pred=y_predicted_test)}")
print(f"Accuracy of basic decision tree w/ train dataset:\t {metrics.accuracy_score(y_true=y_train, y_pred=y_predicted_train)}")

In [ ]:
# Save the decition tree as a png

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(
        decision_tree, 
        filled=True,
        feature_names=features, 
        rounded=True, 
        class_names=wtype,
    )

fig.savefig("decision_tree.png")

In [ ]:
# Plot the importance of each feature
index_sort = np.argsort(decision_tree.feature_importances_)
forest_importances = pd.Series(decision_tree.feature_importances_[index_sort], index=np.array(features)[index_sort])

fig, ax = plt.subplots()
forest_importances.plot.bar( ax=ax)
ax.set_title("Feature importances")
ax.set_ylabel("Feature importance")
fig.tight_layout()

#### 11. Fit a boosted decision tree classifier (1 Mark)


* Create a `sklearn.tree.GradientBoostingClassifier` and set the maximum tree depth to 4 and n_estimators to 20.
* Fit the boosted decision tree classifier
* Predict the response and pring the accuracy for the test and train datasets

In [ ]:
# HYPER-PARAMETERS
NUM_ESTIMATORS = 20
MAX_DEPTH = 4
LEARNING_RATE = 0.1
LOSS_FUNCTION = "log_loss"

# Define bvoosted decision tree object
boosted_decision_tree = GradientBoostingClassifier(
    loss=LOSS_FUNCTION,
    learning_rate=LEARNING_RATE,
    n_estimators=NUM_ESTIMATORS,
    max_depth=MAX_DEPTH,
)

# Fit the boosted decision tree
boosted_decision_tree.fit(x_train, y_train)

# Compute predicted classes for boosted decision tree
y_predicted_test = boosted_decision_tree.predict(x_test)
y_predicted_train = boosted_decision_tree.predict(x_train)

# Print accuracy of the model
print(f"Accuracy of basic decision tree w/ test dataset:\t {metrics.accuracy_score(y_true=y_test, y_pred=y_predicted_test)}")
print(f"Accuracy of basic decision tree w/ train dataset:\t {metrics.accuracy_score(y_true=y_train, y_pred=y_predicted_train)}")

#### 12. Fit a Random forest decision tree classifier (1 Mark)

* Use the splitted dataset defined in exercise 10.
* Create a `sklearn.tree.RandomForestClassifier` and set the maximum tree depth to 4 and n_estimators to 100.
* Use a low number of max features per tree, 3 to 5
* Fit the random forest decision tree classifier
* Predict the response for the test dataset
* Print the accuracy
* Discuss the results of the accuracy for the 3 methods (DT, BDT, RF). How could you improve on the results?

In [ ]:
# HYPER-PARAMETERS
NUM_ESTIMATORS = 100
MAX_DEPTH = 3
LEARNING_RATE = 0.1
CRITERION = "gini"
MAX_FEATURES = 5

random_forest = RandomForestClassifier(
    criterion=CRITERION,
    n_estimators=NUM_ESTIMATORS,
    max_depth=MAX_DEPTH,
    max_features=MAX_FEATURES
)

# Fit the random forest
random_forest.fit(x_train, y_train)

# Compute predicted classes for random forest
y_predicted_test = random_forest.predict(x_test)
y_predicted_train = random_forest.predict(x_train)

# Print accuracy of the model
print(f"Accuracy of basic decision tree w/ test dataset:\t {metrics.accuracy_score(y_true=y_test, y_pred=y_predicted_test)}")
print(f"Accuracy of basic decision tree w/ train dataset:\t {metrics.accuracy_score(y_true=y_train, y_pred=y_predicted_train)}")

**A**
- BDT performed better than all on training sample, but shows sign of overtraining
- RF performed marginally better than DT on testing sample, and has little overtraining
- Performace needs to be tuned by doing a hyperparameter optimisation

# Hyper parameter optimisation (bonus)

We can use built in sklearn functionality to make the search of hyperparameters easier. We can use =GridSearchCV= though we will only introduce CrossValidation next week. 

In [ ]:
from sklearn.model_selection import GridSearchCV

def plot_larning_rate_matrix(results, ada_dic):
    # Desired: param_learning_rate, param_n_estimators, mean_test_score
    scores = np.asarray(results["mean_test_score"]).reshape(4,6)

    fig, ax = plt.subplots(1, 1)
    im = ax.imshow(scores, cmap="gray")
    ax.set(
        xticks=np.arange(0,6),
        yticks=np.arange(0,4), 
        yticklabels=ada_dic["learning_rate"],
        xticklabels=ada_dic["n_estimators"]
    )
    ax.set_title("Learning rate matrix")
    ax.set_xlabel("learning rate")
    ax.set_ylabel("number of estimatiors")
    for i in range(4):
        for j in range(6):
            text = ax.text(
                j, i, 
                f"{scores[i, j]:.2f}",
                ha="center", 
                va="center", 
                color="b"
            )

    fig.colorbar(im)
    plt.show()

Use the following dictionary to run a scan over AdaBoostClassifer. Note that `n_jobs` can be used to employ multiple CPUs on your PC.

Get the best fit parameters from your search. Hint: check the methods attached to the search object. `best_estimator_` is particularly useful for this.

Calculate the accuracy as above.

Try other algorithms (GradBoosting, RandomForests)

In [ ]:
ada_dic={
    "n_estimators":[10,50,100,200,500,1000],
    "learning_rate": [.05,.1,.5,1],
    }

# Define random forest object
ada_classifier = AdaBoostClassifier()

grid = GridSearchCV(ada_classifier, ada_dic, n_jobs=5)
grid.fit(x_train, y_train)

print(f"The best score is \t{grid.best_score_}")
print(f"The best parameters are \t{grid.best_params_}")

results = pd.DataFrame.from_dict(grid.cv_results_)

In [ ]:
# Plot matrix of scores for each combination of hyperparameters
plot_larning_rate_matrix(results, ada_dic)